In [13]:
import numpy
import matplotlib.pyplot as pyplot

from numpy.random import RandomState
from sklearn.datasets import fetch_olivetti_faces

dataset = fetch_olivetti_faces(shuffle = True, random_state = RandomState(127))

# Load and condition dataset

In [14]:
faces = dataset.data

print('faces: %s' % str(faces.shape)) # |samples| x |features|

# global centering
faces_mean = faces.mean(axis = 0)
faces_centered = faces - faces_mean

# local centering
faces_centered -= faces_centered.mean(axis = 1).reshape(faces.shape[0], -1)

faces_centered.shape # |samples| x |features|

split = int(0.25*len(faces))

test_faces = faces_centered[:split]
test_targets = dataset.target[:split]

train_faces = faces_centered[split:]
train_targets = dataset.target[split:]

faces: (400, 4096)


# Perform Eigen Analysis

"The eigenvectors are derived from the covariance matrix of the probability distribution over the high-dimensional vector space of face images. The eigenfaces themselves form a basis set of all images used to construct the covariance matrix. This produces dimension reduction by allowing the smaller set of basis images to represent the original training images. Classification can be achieved by comparing how faces are represented by the basis set." __- Wikipedia__

In other words, PCA is a linear transformation of data to a new coordinate system. Converting large data sets to a coordinate system has many advantages and applications for computer vision. Large images can be represented as smaller vectors which is why PCA is a technique in auto-encoding.

We can match one face with another by projecting a face into the manifold and comparing the distance to a traning face projected into the manifold.

Since Euclidean distance is N dimensional, we can use this as our cost function. We can also rank the traning set face in order of how closely they match the test face.

## The Procedure
1. Load a training set of faces
    * compute the mean of these faces
    * compute the centered set of faces
2. Compute the covariance matrix using the centred set of faces
3. Compute the eigenvectors fo the covariance matrix using Singular Value Decomposition (SVD)

## The Speed-up
The procedure can become computationally intensive if the wrong approach is taken when generating the covariance matrix of the set of centred faces. Consider that the number of features (pixels) will likely be much larget than the number of samples (faces).

* Let $A$ be the set of centred faces, samples by features
* Let $C$ be the covariance matrix of $A$, such that $C = A \cdot A^T$
* Let $L$ be the surrogate of matrix $C$, such that $L = A^T \cdot A$

Matrix $C$ will yeild a size of __features by features__

Matrix $L$ will yeild a size of __samples by samples__

Calculating $C$ directly is computationally expensive since the number of features is significantly larger than then number of samples. There is however a faster way which will be explained below.


## The Math
* Let $v_C$ be the set of eigenvectors of matrix $C$
* Let $\lambda_C$ be the set of eigenvalues of matrix $C$
* Let $v_L$ be the set of eigenvectors of matrix $L$
* Let $\lambda_L$ be the set of eigenvalues of matrix $L$

__Proof__

Consider the eigenvector $v_L$ of $L = A^T \cdot A$ such that,

$L \cdot v_L = \lambda_L \cdot v_L$

$(A^T \cdot A) \cdot v_L = \lambda_L \cdot v_L$

Premultiplying both sides by $A$, produces,

$A \cdot (A^T \cdot A \cdot v_L) = A \cdot (\lambda_L \cdot v_L)$

$(A \cdot A^T) \cdot A \cdot v_L = A \cdot (\lambda_L \cdot v_L)$

$C \cdot (A \cdot v_L) = \lambda_L \cdot (A \cdot v_L)$

Therefore,
* $A \cdot v_L$ is an eigenvector of $C$, and,
* $\lambda_C$ is an eigenvalue of $C$

Therefore,
* $v_C = A \cdot v_L$
* $\lambda_C = \lambda_L$

This concludes that $v_C = A \cdot v_L$, which is very inexpensive to compute since the number of samples is significantly smaller than features.

## Compute surrogate covariance matrix

In [15]:
A = train_faces
print('A: %s' % str(A.shape)) # |samples\ x |features|
L = A.dot(A.T)
print('L: %s' % str(L.shape)) # |samples| x |samples|

A: (300, 4096)
L: (300, 300)


## Compute strong eigen vectors
We can auto-encode at this step by discarding any eigenvectors which are not strong basis vectors, ie they are nearly parallel to another basis. This is a compression technique that sacrifices accuracy for efficiency.

In [16]:
(L_eigenvalues, L_eigenvectors) = numpy.linalg.eig(L)
L_eigenvectors_strong = L_eigenvectors[:,numpy.array([True if (x > 1) else False for x in L_eigenvalues])]
print('L eigenvectors: %s' % str(L_eigenvectors_strong.shape)) # |samples| x |strong samples|

L eigenvectors: (300, 296)


## Compute eigenfaces

In [17]:
C_eigenvectors = A.T.dot(L_eigenvectors_strong).T # eigenfaces
eigenfaces = C_eigenvectors
print('eigenfaces: %s' % str(eigenfaces.shape)) # |samples| x |features|

eigenfaces: (296, 4096)


## Project faces into eigenface space
This is the last step of training our model.

In [18]:
# Project all training data into eigenface space
train_faces_projected = numpy.vstack([eigenfaces.dot(train_face) for train_face in train_faces])
train_faces_projected.shape # |samples| x |strong samples|

(300, 296)

# Perform test on faces outside our model

In [19]:
# Match test faces with training

correct = 0.0
trails = len(test_faces)

for test_index in range(trails):
    
    test_face = test_faces[test_index]
    test_target = test_targets[test_index]
    
    test_face_projected = eigenfaces.dot(test_face) # Note: faces are already normalized
    
    distances = numpy.array([((test_face_projected - train_face_projected)**2).sum() for train_face_projected in train_faces_projected])
    
    guess_index = distances.argmin()
    guess_face = train_faces[guess_index]
    guess_target = train_targets[guess_index]
    
    is_correct = (test_target == guess_target)
    
    if (is_correct):
        correct += 1.0
    
    # Show some examples of matching...
    if (test_index < 10):
        # Test face...
        pyplot.subplot(1,2,1)
        pyplot.title('Test face')
        pyplot.imshow(
            test_face.reshape([64, 64]),
            cmap = pyplot.cm.gray,
            interpolation = 'nearest'
        )
        # Guess face...
        pyplot.subplot(1,2,2)
        pyplot.title('Guess face [%s match]' % ('Correct' if is_correct else 'Incorrect'))
        pyplot.imshow(
            guess_face.reshape([64, 64]),
            cmap = pyplot.cm.gray,
            interpolation = 'nearest'
        )
        # Show...
        pyplot.show()

print '%d%% accuracy' % int(100.0*correct/trails)
print '... amongst %d unique people' % int(len(frozenset(dataset.target)))

76% accuracy
... amongst 40 unique people
